In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy import stats

from one.api import ONE
from brainbox.io.one import SessionLoader

from iblnm.config import *
from iblnm.util import protocol2type, _load_event_times
from iblnm.resp import get_responses

# FIXME: this import depends on config being imported first
from iblphotometry import io, metrics

## Pick a sessions by querying Alyx
(See below for selecting sessions using df_session from a previous query)

In [2]:
one = ONE()

In [3]:
# Pick a subject
subject = 'ZFM-08828'

df_sessions = pd.DataFrame(one.alyx.rest('sessions', 'list', project='ibl_fibrephotometry', subject=subject, dataset='photometry.signal.pqt')).rename(columns={'id':'eid'})
df_sessions.head()

,eid,subject,start_time,number,lab,projects,url,task_protocol
0,fc5d4d0f-c3aa-49ff-a0a5-e89cdd7d9466,ZFM-08828,2025-09-08T13:57:04.395523,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
1,1b2d4f35-b49e-4cd9-82d7-b0b4243e8299,ZFM-08828,2025-09-05T14:07:40.933675,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
2,c3f82159-4187-45eb-9b51-d627aba29c74,ZFM-08828,2025-09-04T15:22:23.395487,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
3,a31782d9-3cea-4315-8c0a-80414de9e232,ZFM-08828,2025-09-03T16:21:49.283018,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0
4,c165deef-3c26-4de6-996e-c21b8eec37da,ZFM-08828,2025-09-02T14:24:39.442487,1,mainenlab,[ibl_fibrephotometry],https://alyx.internationalbrainlab.org/session...,_iblrig_tasks_trainingChoiceWorld8.29.0


In [11]:
# Pick a session
session_i = 3
session = df_sessions.iloc[session_i]
session

eid                           a31782d9-3cea-4315-8c0a-80414de9e232
subject                                                  ZFM-08828
start_time                              2025-09-03T16:21:49.283018
number                                                           1
lab                                                      mainenlab
projects                                     [ibl_fibrephotometry]
url              https://alyx.internationalbrainlab.org/session...
task_protocol              _iblrig_tasks_trainingChoiceWorld8.29.0
Name: 3, dtype: object

## Load photometry and task data

In [12]:
# Get task events
session = _load_event_times(session, one)

(S3) /home/davide/Downloads/ONE/alyx.internationalbrainlab.org/mainenlab/Subjects/ZFM-08828/2025-09-03/001/alf/task_00/_ibl_trials.table.pqt


In [22]:
# Get photometry data
df_photometry = one.load_dataset(id=session['eid'], dataset='photometry.signal.pqt')

# Get times for the task period
timings = [col for col in session.index if col.endswith('_times')]
t0 = np.concatenate(session[timings].values).min()
t1 = np.concatenate(session[timings].values).max()
i0 = df_photometry['times'].searchsorted(t0)
i1 = df_photometry['times'].searchsorted(t1)

# Restrict to task period and convert to a more workable format
signals = io.from_ibl_dataframe(df_photometry.iloc[i0:i1])

# Pull out gcamp and iso dataframes
gcamp = signals['GCaMP']
iso = signals['Isosbestic']

# Get fiber locations and ROIs
locations = one.load_dataset(id=session['eid'], dataset='photometryROI.locations.pqt').reset_index()
print(locations)

  ROI      fiber brain_region
0  G1  fiber_VTA          VTA


In [23]:
# Pick an ROI
brain_region = 'VTA'
fiber = locations.query('brain_region == @brain_region')
assert len(fiber) == 1  # just one fiber at a time for now
roi = fiber['ROI'].iloc[0]

In [12]:
# Get responses to task events
events = ['cue', 'movement', 'reward', 'omission']
psths = []
for event in events:
    responses, tpts = get_responses(gcamp[roi], session[f'{event}_times'])
    psths.append(responses)

In [11]:
%matplotlib qt

fig = plt.figure(figsize=(12, 8))
grid = gridspec.GridSpec(2, 3)

ax2 = fig.add_subplot(grid[1, 0])
ax3 = fig.add_subplot(grid[1, 1])
ax4 = fig.add_subplot(grid[1, 2])
ax1 = fig.add_subplot(grid[0, :])

ax1.plot(gcamp[roi])
ax1.plot(iso[roi], color='gray')
ax1.set_ylabel('Signal (a.u.)')
ax1.set_title(f"{session['subject']} {session['start_time']} {session['task_protocol']}")

colors = [EVENT2COLOR[event] for event in events]
for event, responses, color, ax in zip(events, psths, colors, [ax2, ax3, ax4, ax4]):
    ax.plot(tpts, responses.mean(axis=0), color=color, label=event)
    ax.plot(tpts, responses.mean(axis=0) - stats.sem(responses, axis=0), ls='--', color=color)
    ax.plot(tpts, responses.mean(axis=0) + stats.sem(responses, axis=0), ls='--', color=color)
    ax.axvline(0, ls='--', color='black', alpha=0.5)
    ax.axhline(0, ls='--', color='gray', alpha=0.5)
    ax.set_xlabel('Time (s)')
    if event != 'omission':
        ax.set_title(event.capitalize())
    ax.ticklabel_format(axis='y', style='sci', scilimits=[-2, 2])

## WIP: Run QC on the photomtery signal 

In [134]:
# Run quality control

# Sampling regularity (we need the full df for this one, but with sample times as the index)
session['qc_n_early_samples'] = metrics.n_early_samples(df_photometry.set_index('times'))

# Number of unique samples in the signal
session['qc_n_unique_samples_gcamp'] = metrics.n_unique_samples(gcamp[roi])

# Signal amplitude
session['qc_deviance_gcamp'] = metrics.median_absolute_deviance(gcamp[roi])  # median amplitude
session['qc_percentile_distance_gcamp'] = metrics.percentile_distance(gcamp[roi], pc=(50, 95))  # amplitude of positive transients
session['qc_percentile_asymmetry_gcamp'] = metrics.percentile_asymmetry(gcamp[roi], pc_comp=95)  # amplitude of positive versus negative transients

# Outliers
session['qc_n_edges_gcamp'] = metrics.n_edges(gcamp[roi])

 ## Pick a session using df_sessions from a previous query

In [16]:
# Load df_sessions
df_sessions = pd.read_parquet('metadata/sessions.pqt')
df_sessions = df_sessions[df_sessions['target'].apply(len) > 0]  # restrict to sessions with a target

# Make a column for target-NM combinations and print a summary of the dataset
df_sessions['target_NM'] = df_sessions.apply(lambda x: '-'.join([x['target'][0], x['NM']]), axis='columns')
print("Number of sessions for each", df_sessions['target_NM'].value_counts())

Number of sessions for each target_NM
LC-NE        358
DR-5HT       345
NBM-ACh      279
VTA-DA       266
SNc-DA       145
MR-5HT        42
SI-ACh        42
SNC-DA        13
NBM-l-ACh      8
PPT-ACh        6
VTA-none       2
MGv-5HT        1
Name: count, dtype: int64


In [18]:
# Pick a target
target_NM = 'VTA-DA'
df_target = df_sessions.query('target_NM == @target_NM')

# Print a detailed breakdown of sessions per mouse
print(df_target.groupby(['subject', 'session_type']).count()['session_n'])

subject    session_type
ZFM-03447  biased          10
           training        26
ZFM-04022  biased          16
           misc             1
           training        10
ZFM-04026  biased          12
           training        22
ZFM-08488  training         7
ZFM-08554  training         7
ZFM-08689  habituation      3
           training         5
ZFM-08818  habituation      3
           training        47
ZFM-08827  habituation      3
           training        48
ZFM-08828  habituation      3
           misc             1
           training        25
ZFM-09044  habituation      3
           training        14
Name: session_n, dtype: int64


In [21]:
# Pick a session
subject = 'ZFM-08827'
session_type = 'training'
session_n = 0  # numerical index for the subject, 0 is the most recent
session = df_target.query('(subject == @subject) & (session_type == @session_type)').iloc[session_n]
print(session[['eid', 'subject', 'target_NM', 'start_time', 'task_protocol']])

eid                 69c74e8d-ba24-4f4f-9718-46087c9731bc
subject                                        ZFM-08827
target_NM                                         VTA-DA
start_time                    2025-08-25T13:30:43.028022
task_protocol    _iblrig_tasks_trainingChoiceWorld8.29.0
Name: 2, dtype: object


Return to "Load photometry and task data"...